# Load Model

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="qwen3-8B", 
    base_url="http://127.0.0.1:1234/v1", 
    api_key=""
)

# Add Message & Tool

In [3]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

claims_data_input = """
- Hospital: RS ABC, Claim ID: 1001, Diagnosis: Cataract, Procedure: Phacoemulsification, Cost: 5M
- Hospital: RS ABC, Claim ID: 1002, Diagnosis: Cataract, Procedure: Phacoemulsification, Cost: 31M
- Hospital: RS ABC, Claim ID: 1003, Diagnosis: Cataract, Procedure: Phacoemulsification, Cost: 31M
- Hospital: RS ABC, Claim ID: 1004, Diagnosis: Cataract, Procedure: Phacoemulsification, Cost: 31M
"""

messages = [
    SystemMessage(
    """
        You are an expert data analyst with a specialization in fraud and anomaly detection. Your task is to rigorously analyze the provided data to identify any suspicious, fraudulent, or anomalous activities.
        The specific context of this analysis is claims from hospitals to BPJS (as the verifier), where potential fraud may occur due to limited verification resources.
        When analyzing, pay close attention to the following indicators and potential anomalies:
        1. Anomalous Disease Distribution: Look for unusual distributions of diseases per month (e.g., one disease dominating 80% of claims). Also, consider external context (e.g., is this the only hospital in a densely populated area?).
        2. Claims Outside of Specialization: Identify high volumes of claims that do not align with the hospital's primary specialities.
        3. Claims Exceeding Facilities: Detect claims for procedures that are not supported by the hospital's registered facilities or equipment.
        4. Medical Record Inconsistencies: Verify the alignment and justification between the diagnoses (primary & secondary) and the medical procedures performed.
        5. Price Markup (Upcoding): Detect significant inflation of costs for procedures or medicines compared to the standard rates (e.g., a cataract surgery claimed at 30M when the normal cost is 10M).

        Your response must be a concise, professional report. Do not use conversational language. Use Indonesian language for the report.
        Your output must strictly follow this format: (Don't include your think steps or analysis process, just the final report)

        Fraud Analysis Report
        --------------------
        1. Fraud / No Fraud ? 
        2. Confidence Level ? (Low / Medium / High)
        3. Key Findings & Indicators
        [Bulleted list of the specific data points, patterns, or outliers that support your
    """
    ),
    HumanMessage(content=f"Please analyze the following claims data for potential fraud or anomalies: {claims_data_input}"),
    AIMessage(content="Thank you for providing the claims data. I will now analyze it for any signs of fraud or anomalies.")
]

prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | model | StrOutputParser()

result = chain.invoke(
    {
    }
)
print(result)


 Let's take a look at each claim individually and then consider them as a group to identify patterns that may indicate fraudulent activity.
</think>

Fraud Analysis Report
--------------------
1. Fraud / No Fraud ? 
   Fraud

2. Confidence Level ? (Low / Medium / High)
   High

3. Key Findings & Indicators
   - Klaim dengan ID 1001 memiliki biaya sebesar 5M, yang sesuai dengan standar tarif untuk prosedur Phacoemulsifikasi pada diagnosis Katarak.
   - Klaim dengan ID 1002, 1003, dan 1004 menunjukkan biaya yang sama (31M), jauh melebihi tarif standar sektor kesehatan untuk prosedur tersebut. 
   - Terdapat ketidaksesuaian signifikan antara nilai biaya yang dilaporkan dan standar harga pasar, menunjukkan indikasi upcoding (penaikan harga secara tidak sah).
   - Pola klaim dengan biaya tinggi berulang pada diagnosis dan prosedur yang sama menunjukkan potensi penyalahgunaan sistem klaim.


# Use Tool & Database Connection

In [10]:
import sys
sys.path.append('..')

from src.tool.tool_registry import ToolRegistry
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(
    model="qwen3-8B", 
    base_url="http://127.0.0.1:1234/v1", 
    api_key=""
)

tool_registry = ToolRegistry()
tools = tool_registry.get_tools()

agent_executor = create_agent(model, tools)

user_input = "Beri saya informasi mengenai RSIA Melinda"

messages = [
    SystemMessage(
    """
        Give me consise and informative answer only. No additional explanations. No thinking steps.
    """
    ),
    HumanMessage(content=user_input)
]

response = agent_executor.invoke({
    "messages": messages
})

print(f"Query: {user_input}")
print(f"Response: {response['messages'][-1].content}")

Query: Beri saya informasi mengenai RSIA Melinda
Response: <think>

</think>

RSIA Melinda adalah rumah sakit spesialis kelas Mother and Child. Lokasinya berada di koordinat lat: -6.906860469056542, long: 107.60326417692049. Fasilitas yang tersedia meliputi apotek, radiologi, laboratorium, instalasi gawat darurat (IGD), area parkir, rawat inap, ambulans, kamar bayi, dan layanan rawat jalan. Spesialisasi yang ditawarkan mencakup poliklinik obstetri dan ginekologi, kardiologi, gastroenterologi, kesehatan anak, neurologi, serta onkologi.
